# APS360 - Classifying Subreddits

Bassam Bibi<br>
Matthew Kwan

## Imports

In [1]:
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

## Load Data

In [23]:
df = pd.read_csv('./data/updated_datav4.csv')
df.head()

,id,post,score,subreddit,title,url,subreddit_name
0,0,NaN,53782,0,"New ""Discovery Mode"" turns video game ""Assassi...",https://www.theverge.com/2018/2/20/17033024/as...,history
1,1,"Hi everyone and especially our students,\r\r\r...",38426,0,We are not here to help you with your End of T...,https://www.reddit.com/r/history/comments/8pw3...,history
2,2,NaN,35982,0,A 1776 excerpt from John Adam's diary where he...,https://founders.archives.gov/documents/Adams/...,history
3,3,NaN,34908,0,Famous Viking warrior burial revealed to be th...,http://www.news.com.au/technology/science/arch...,history
4,4,NaN,34197,0,"3,000-year-old underwater castle discovered in...",https://inhabitat.com/3000-year-old-underwater...,history


### Splitting Function for Punctuations

In [24]:
def split_post(post):
    # separate punctuations
    post = post.replace(".", " . ") \
                 .replace(",", " , ") \
                 .replace(";", " ; ") \
                 .replace("?", " ? ") 

    return post.split() # Returns each word as an index in a list


### Download GloVe Vectors

In [25]:
# Download pre-trained glove vectors

glove = torchtext.vocab.GloVe(name="6B", dim=50, max_vectors=10000) 

### Function the Split Data into Train, Test and Valid

In [26]:
def get_post_words(glove_vector):
    train, valid, test = [], [], []
    for i, line in enumerate(df.post):
        if i%3 == 0:
            
            # If there is no text content in the post, then use the title of the post instead
            
            if line is np.nan:
                post = df.title[i]
            else:
                post = line
            
            idxs = [glove_vector.stoi[w]        # lookup the index of word
                    for w in split_post(post)
                    if w in glove_vector.stoi] # keep words that has an embedding
            if not idxs: # ignore posts without any word with an embedding
                continue
            idxs = torch.tensor(idxs) # convert list to pytorch tensor
            label = torch.tensor(df.subreddit[i]).long() #IMPORTANT, need to convert this to numerical category
            if i % 5 < 3:
                train.append((idxs, label))
            elif i % 5 == 4:
                valid.append((idxs, label))
            else:
                test.append((idxs, label))
    return train, valid, test

train,valid,test = get_post_words(glove)

In [27]:
print("Train Length: {} Val Length: {} Test Length: {}".format(len(train),len(valid),len(test)))

Train Length: 2497 Val Length: 836 Test Length: 833


## Batching

In [28]:
import random

class PostBatcher:
    def __init__(self, posts, batch_size=32, drop_last=False):
        # store posts by length
        self.posts_by_length = {}
        for words, label in posts:
            # compute the length of the post
            wlen = words.shape[0]
            # put the posts in the correct key inside self.posts_by_length
            if wlen not in self.posts_by_length:
                self.posts_by_length[wlen] = []
            self.posts_by_length[wlen].append((words, label),)
         
        #  create a DataLoader for each set of posts of the same length
        self.loaders = {wlen : torch.utils.data.DataLoader(
                                    posts,
                                    batch_size=batch_size,
                                    shuffle=True,
                                    drop_last=drop_last) # omit last batch if smaller than batch_size
            for wlen, posts in self.posts_by_length.items()}
        
    def __iter__(self): # called by Python to create an iterator
        # make an iterator for every post length
        iters = [iter(loader) for loader in self.loaders.values()]
        while iters:
            # pick an iterator (a length)
            im = random.choice(iters)
            try:
                yield next(im)
            except StopIteration:
                # no more elements in the iterator, remove it
                iters.remove(im)

With our batching function working, let's define our data loaders. Let's start with a batch-size of 16:

In [36]:
train_loader = PostBatcher(train,batch_size=16,drop_last=True)
valid_loader = PostBatcher(valid,batch_size=16,drop_last=True)

In [30]:
for i, (posts, labels) in enumerate(train_loader):
    if i > 5: break
    print(posts.shape, labels.shape)

torch.Size([16, 19]) torch.Size([16])
torch.Size([16, 8]) torch.Size([16])
torch.Size([16, 2]) torch.Size([16])
torch.Size([16, 27]) torch.Size([16])
torch.Size([16, 9]) torch.Size([16])
torch.Size([16, 3]) torch.Size([16])


In [31]:
train_loader

## Preliminary Model

In [32]:
class RedditLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes=13):
        super(RedditLSTM, self).__init__()
        self.emb = nn.Embedding.from_pretrained(glove.vectors)
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True, num_layers=3)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Look up the embedding
        x = self.emb(x)
        # Set an initial hidden state and cell state
        h0 = torch.zeros(3, x.size(0), self.hidden_size)
        c0 = torch.zeros(3, x.size(0), self.hidden_size)
        # Forward propagate the LSTM
        out, _ = self.rnn(x, (h0, c0))
        # Pass the output of the last time step to the classifier
        out = self.fc(out[:, -1, :])
        return out

## Training 

In [33]:
# Function to calculate the accuracy of the results. 
# Returns: # of Correct Predictions / Total # of Predictions

def get_accuracy(model, data_loader, find_loss=False):
  
    criterion = nn.CrossEntropyLoss()

    correct, total = 0, 0
    for posts, labels in data_loader:
        output = model(posts)
        if find_loss == True:
            loss = criterion(output,labels)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += labels.shape[0]
   
    if find_loss == False:
        return correct / total
    else:
        return correct / total,loss

In [34]:
def train_rnn_network(model, train, valid, num_epochs=200, learning_rate=1e-4):
    criterion = nn.CrossEntropyLoss() # Need Cross Entropy for Multi-Classification
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    losses, train_acc, valid_acc, val_losses = [], [], [], []
    epochs = []
    for epoch in range(num_epochs):
        for posts, labels in train:
            optimizer.zero_grad()
            pred = model(posts)
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()
        losses.append(float(loss))

        epochs.append(epoch)
        train_acc.append(get_accuracy(model, train_loader))
        val_acc,val_loss = get_accuracy(model, valid_loader, find_loss=True)
        val_losses.append(val_loss)
        valid_acc.append(val_acc)
        if epoch%50 == 0:
            print("Epoch %d; Train Loss %f; Validation Loss %f; Train Acc %f; Val Acc %f" % (
                  epoch, loss, val_loss, train_acc[-1], valid_acc[-1]))
    # plotting
    plt.title("Training Curve")
    plt.plot(losses, label="Train")
    plt.plot(val_losses, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(epochs, train_acc, label="Train")
    plt.plot(epochs, valid_acc, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()

In [35]:
model = RedditLSTM(50,50)
train_rnn_network(model,train_loader,valid_loader,num_epochs=1000)

Epoch 0; Train Loss 2.510890; Validation Loss 2.440137; Train Acc 0.170213; Val Acc 0.163564
Epoch 50; Train Loss 1.680194; Validation Loss 1.483324; Train Acc 0.323138; Val Acc 0.315160
Epoch 100; Train Loss 1.483804; Validation Loss 1.992002; Train Acc 0.424202; Val Acc 0.429521
Epoch 150; Train Loss 1.444066; Validation Loss 1.347528; Train Acc 0.539894; Val Acc 0.545213
Epoch 200; Train Loss 1.643146; Validation Loss 1.256178; Train Acc 0.598404; Val Acc 0.586436
Epoch 250; Train Loss 0.437440; Validation Loss 0.604233; Train Acc 0.720745; Val Acc 0.715426
Epoch 300; Train Loss 0.576510; Validation Loss 1.306043; Train Acc 0.744681; Val Acc 0.744681
Epoch 350; Train Loss 0.749690; Validation Loss 0.306805; Train Acc 0.815160; Val Acc 0.813830
Epoch 400; Train Loss 0.392854; Validation Loss 0.585332; Train Acc 0.844415; Val Acc 0.837766
Epoch 450; Train Loss 0.326381; Validation Loss 0.209531; Train Acc 0.845745; Val Acc 0.867021
Epoch 500; Train Loss 0.351617; Validation Loss 1.289

KeyboardInterrupt: 

In [55]:
# Create Test Loader
test_loader = PostBatcher(test,batch_size=16,drop_last=True)

In [56]:
test_acc = get_accuracy(model,test_loader)
print("Our accuracy on our test set was: {}".format(test_acc))

Our accuracy on our test set was: 0.546875


## Tuning Hyperparameters
First, let's try increasing the batch size to 32:

In [ ]:
train_loader = PostBatcher(train,batch_size=32,drop_last=True)
valid_loader = PostBatcher(train,batch_size=32,drop_last=True)
model = RedditLSTM(50,50)
train_rnn_network(model,train_loader,valid_loader,num_epochs=1000)

In [ ]:
train_loader = PostBatcher(train,batch_size=64,drop_last=True)
valid_loader = PostBatcher(train,batch_size=64,drop_last=True)
model = RedditLSTM(50,50)
train_rnn_network(model,train_loader,valid_loader,num_epochs=1000)

In [ ]:
train_loader = PostBatcher(train,batch_size=64,drop_last=True)
valid_loader = PostBatcher(train,batch_size=64,drop_last=True)
model = RedditLSTM(50,50)
train_rnn_network(model,train_loader,valid_loader,learning_rate = 0.5e-3,num_epochs=1000)

In [ ]:
train_loader = PostBatcher(train,batch_size=64,drop_last=True)
valid_loader = PostBatcher(train,batch_size=64,drop_last=True)
model = RedditLSTM(50,126)
train_rnn_network(model,train_loader,valid_loader,learning_rate = 0.5e-3,num_epochs=1000)

In [ ]:
train_loader = PostBatcher(train,batch_size=64,drop_last=True)
valid_loader = PostBatcher(train,batch_size=64,drop_last=True)
model = RedditLSTM(50,256)
train_rnn_network(model,train_loader,valid_loader,learning_rate = 0.5e-3,num_epochs=1000)